In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
os.chdir('data')
os.listdir()

## Loading the datasets

In [ ]:
data = {}

for file in os.listdir():
    data[file] = pd.read_csv(file)

In [ ]:
os.chdir('../')

## Patient medical trajectory

In [ ]:
# select random patient from the patient directory
patient_id = data['patients.csv'].sample(n=1).reset_index(drop = True)['Id']

In [ ]:
patient_encounters = data['encounters.csv'][data['encounters.csv']['PATIENT'] == patient_id[0]]
patient_conditions = data['conditions.csv'][data['conditions.csv']['PATIENT'] == patient_id[0]]
patient_medications =  data['medications.csv'][data['medications.csv']['PATIENT'] == patient_id[0]]
patient_procedures = data['procedures.csv'][data['procedures.csv']['PATIENT'] == patient_id[0]]

### Visualizing patients conditions

In [ ]:
dates = pd.to_datetime(patient_conditions['START'].reset_index(drop = True))
description = patient_conditions['DESCRIPTION'].reset_index(drop = True)
birthdate = data['patients.csv'][data['patients.csv']['Id'] == patient_id[0]].reset_index(drop = True)['BIRTHDATE'][0]

In [ ]:
levels = np.tile([-4, 4, -3, 3, -2, 2],
                 int(np.ceil(len(dates)/6)))[:len(dates)]


fig, ax = plt.subplots(figsize =(10,7))
ax.yaxis.set_visible(False)
ax.spines[["left", "top", "right"]].set_visible(False)
ax.plot(dates,np.zeros_like(dates),"-o",markerfacecolor = "w")
ax.vlines(dates,0,levels)

for d, l, r in zip(dates, levels, description):
    ax.annotate(r, xy=(d, l),
                xytext=(-3, np.sign(l)*3), textcoords="offset points",
                horizontalalignment="center",
                verticalalignment="bottom" if l > 0 else "top",fontsize = 10)

plt.title('Patient Medical Conditions: Timeline \n Birthdate:{}'.format(birthdate),y = 1.05)
plt.savefig('medical_timeline.pdf')
plt.show()



### List of medications prescribed for each condition:

In [ ]:
cond_med = pd.DataFrame({'Condition':patient_medications.groupby(['REASONDESCRIPTION','DESCRIPTION'])['DESCRIPTION'].count().index.get_level_values(0),
                       'Medication':patient_medications.groupby(['REASONDESCRIPTION','DESCRIPTION'])['DESCRIPTION'].count().index.get_level_values(1)
                      }).reset_index(drop = True)

cond_med

### Procedures undergone by patient:

In [ ]:
procedure = patient_procedures[['DATE','DESCRIPTION','REASONDESCRIPTION']]
procedure['DATE'] = pd.to_datetime(procedure['DATE']).dt.year
procedure.rename(columns = {'REASONDESCRIPTION':'Condition','DATE':'Year','DESCRIPTION':'Procedure'},inplace = True)


procedure

## What are the most common conditions?

### Top 10 conditions by number of cases

In [ ]:
condition_counts = pd.DataFrame({'condition': data['conditions.csv'].groupby('DESCRIPTION')['DESCRIPTION'].count().index,'cases':data['conditions.csv'].groupby('DESCRIPTION')['DESCRIPTION'].count()}).reset_index(drop = True)
condition_counts.sort_values(by = 'cases',ascending = False)[0:5]

In [ ]:
p = sns.barplot(data = condition_counts.sort_values(by = 'cases',ascending = False)[0:10],x = 'cases',y = 'condition',orient = 'h')
plt.xlabel(' Total Number of Cases')
plt.ylabel('Condition')
plt.title('Top ten conditions by total number of cases')
plt.savefig('top_ten_total.pdf',bbox_inches = 'tight')
plt.show()

### Top 10 conditions by number of people affected

In [ ]:
condition_cases = data['conditions.csv'].groupby(['DESCRIPTION','PATIENT'],as_index = False)[['PATIENT']].count()
condition_df = pd.DataFrame({'condition':condition_cases.groupby('DESCRIPTION')['PATIENT'].count().index,
                            'nr_affected':condition_cases.groupby('DESCRIPTION')['PATIENT'].count()}).reset_index(drop = True)
condition_df['percentage'] = np.round(condition_df['nr_affected']/len(data['patients.csv']),2)

In [ ]:
p = sns.barplot(data = condition_df.sort_values(by = 'percentage',ascending = False)[0:10],x = 'percentage',y = 'condition',orient = 'h')
p.set_xticklabels(['0 %','10 %','20 %','30 %','40 %','50 %','60 %'])
plt.xlabel('Percentage of people affected')
plt.ylabel('Condition')
plt.title('Top ten conditions by percentage of people affected')
plt.savefig('top_ten_people.pdf',bbox_inches = 'tight')
plt.show()

In [ ]:
condition_df.sort_values(by = 'percentage',ascending = False)[0:10]

## How are the most common conditions treated?

In [ ]:
top_list = list(condition_df.sort_values(by = 'percentage',ascending = False)[0:3]['condition'])

### Medications

In [ ]:
top_medications = data['medications.csv'][data['medications.csv']['REASONDESCRIPTION'].isin(top_list)]

In [ ]:
med_df = pd.DataFrame(
    {'condition':top_medications.groupby(['REASONDESCRIPTION','DESCRIPTION'])[['DESCRIPTION']].count().index.get_level_values(0) ,
     'medication':top_medications.groupby(['REASONDESCRIPTION','DESCRIPTION'])[['DESCRIPTION']].count().index.get_level_values(1),
     'prescribed':top_medications.groupby(['REASONDESCRIPTION','DESCRIPTION'])['DESCRIPTION'].count()}).reset_index(drop = True)



In [ ]:
med_df = med_df.merge(condition_counts,on ='condition')
med_df['percentage'] = np.round(med_df['prescribed']/med_df['cases'],2)

In [ ]:
med_df

### Procedures

In [ ]:
top_procedures = data['procedures.csv'][data['procedures.csv']['REASONDESCRIPTION'].isin(top_list)]

In [ ]:
proc_df = pd.DataFrame(
    {'condition':top_procedures.groupby(['REASONDESCRIPTION','DESCRIPTION'])[['DESCRIPTION']].count().index.get_level_values(0) ,
     'procedure':top_procedures.groupby(['REASONDESCRIPTION','DESCRIPTION'])[['DESCRIPTION']].count().index.get_level_values(1),
     'counts':top_procedures.groupby(['REASONDESCRIPTION','DESCRIPTION'])['DESCRIPTION'].count()}).reset_index(drop = True)

proc_df = proc_df.merge(condition_counts,on ='condition')
proc_df['percentage'] = np.round(proc_df['counts']/proc_df['cases'],2)


proc_df

### Careplans

In [ ]:
top_careplans = data['careplans.csv'][data['careplans.csv']['REASONDESCRIPTION'].isin(top_list)]

In [566]:
care_df = top_careplans.groupby(['REASONDESCRIPTION','DESCRIPTION'])[['DESCRIPTION']].count()

In [ ]:
care_df

### Are there any other patterns among these condtions?

In [ ]:
# Merging the top three conditions with age of patients
common_age = pd.merge(data['conditions.csv'][data['conditions.csv']['DESCRIPTION'].isin(top_list)],
         data['patients.csv'][['Id','BIRTHDATE','PREFIX']],left_on = 'PATIENT',right_on = 'Id')

In [ ]:
common_age['AGE'] = pd.to_datetime(common_age['START']).dt.year - pd.to_datetime(common_age['BIRTHDATE']).dt.year
common_age.replace({'PREFIX':{'Mrs.':'FEMALE','Ms.':'FEMALE','Mr.':'MALE'}},inplace = True)
common_age.rename(columns = {'PREFIX':'Gender','DESCRIPTION':'Condition'},inplace = True)

####  Distribution of condition by age

In [ ]:
sns.histplot(data = common_age, x = 'AGE',hue = 'Condition',element = 'poly')
plt.title('Distribution of condition by age')
plt.savefig('dist_age.pdf')
plt.show()

#### Distribution of condition by age and gender

In [ ]:
sns.displot(data = common_age, x = 'AGE',hue = 'Condition',col = 'Gender',kind = 'hist')
plt.title('Distribution of conditions by age and gender',loc = 'left', x = -0.60,y = 1.10,fontsize = 15)
plt.savefig('cond_age_gender.pdf',bbox_inches = 'tight')
plt.show()

#### Number of cases by gender

In [ ]:
s = sns.displot(data = common_age, x = 'Condition',hue = 'Gender',multiple = 'dodge',shrink = 0.5)
s.set_xticklabels(['Sinusitis','Pharyngitis','Bronchitis'])
plt.title('Number of cases by gender')
plt.savefig('nr_gender.pdf',bbox_inches = 'tight')
plt.show()